In [5]:
import os
import sys
sys.path.append('/Users/daniel.tomes/Software/spark-2.1.0-bin-hadoop2.6/python/lib/py4j-0.10.4-src.zip')

In [6]:
from email.parser import Parser
from textblob import TextBlob
from pyspark import SparkContext
from pyspark import SparkConf

TypeError: namedtuple() missing 3 required keyword-only arguments: 'verbose', 'rename', and 'module'

In [2]:
sc

''

# Read A Single File and Parse it

In [3]:
file_to_read = 'data/enron_emails/maildir/lay-k/inbox/10.'

In [5]:
with open(file_to_read, "r") as f:
    data = f.read()

In [6]:
email = Parser().parsestr(data)

In [20]:
print("\nMessage-ID: ", email['Message-ID'])


Message-ID:  <1122139.1075845189628.JavaMail.evans@thyme>


In [8]:
print("\nTo: " , email['to'])


To:  kenneth.lay@enron.com


In [10]:
print("\n From: ", email['from'])


 From:  beau@layfam.com


In [11]:
print("\n Subject: " , email['subject'])


 Subject:  LOC update


In [12]:
print("\n \n Body: ", email.get_payload())


 
 Body:  
PW line has a deficit of about $2.1 million.  We are on day 3.   We might have to fund the account around $2 million if it doesn't cure itself by  tomorrow.  Can we use the Enron line again now that it is paid-off or would  you rather use one of the other lines?  BA has approximately $5.9  million available, which has the most availability of all the lines.
 
Please advise.
 
- Beau


In [15]:
bodyBlob = TextBlob(email.get_payload())

In [139]:
bodyBlob.sentiment

NameError: name 'bodyBlob' is not defined

In [ ]:
bod

# Read a Subset of Emails & Parse Them

In [3]:
import os

In [4]:
baseDir = 'data/enron_emails/maildir/lay-k'

Let's make sure we have the directory looping correct os we can get all the files from each sub-dir

In [19]:
for directory, subdirectory, filenames in os.walk(baseDir):
    print(directory, subdirectory, len(filenames))

data/enron_emails/maildir/lay-k ['_sent', 'all_documents', 'business', 'calendar', 'compaq', 'deleted_items', 'discussion_threads', 'elizabeth', 'enron', 'family', 'inbox', 'notes_inbox', 'sec_panel', 'sent', 'sent_items'] 0
data/enron_emails/maildir/lay-k/_sent [] 261
data/enron_emails/maildir/lay-k/all_documents [] 1127
data/enron_emails/maildir/lay-k/business [] 2
data/enron_emails/maildir/lay-k/calendar [] 8
data/enron_emails/maildir/lay-k/compaq [] 1
data/enron_emails/maildir/lay-k/deleted_items [] 1126
data/enron_emails/maildir/lay-k/discussion_threads [] 910
data/enron_emails/maildir/lay-k/elizabeth [] 35
data/enron_emails/maildir/lay-k/enron [] 5
data/enron_emails/maildir/lay-k/family [] 8
data/enron_emails/maildir/lay-k/inbox [] 1373
data/enron_emails/maildir/lay-k/notes_inbox [] 799
data/enron_emails/maildir/lay-k/sec_panel [] 3
data/enron_emails/maildir/lay-k/sent [] 266
data/enron_emails/maildir/lay-k/sent_items [] 13


In [5]:
def email_analysis(inputfile, to_message_id, to_email_list, from_email_list, email_body):
    with open(inputfile, "r", encoding='utf-8', errors='ignore') as f:
        data = f.read()
    
    email = Parser().parsestr(data)
    
    to_message_id.append(email['Message-ID'])
    to_email_list.append(email['to'])
    from_email_list.append(email['from'])
    
    email_body.append(email.get_payload())

In [6]:
email_id = []
email_to_list = []
email_from_list = []
email_body = []

for directory, subdirectory, filenames in os.walk(baseDir):
    for filename in filenames:
        email_analysis(os.path.join(directory,filename), email_id, email_to_list, email_from_list, email_body)

Let's check the length of each list to make sure none of them got skewed.

In [7]:
print("Email IDs present: %s" % len(email_id))
print("Email Tos present: %s" % len(email_to_list))
print("Email Froms present: %s" % len(email_from_list))
print("Email Bodies present: %s" % len(email_body))

Email IDs present: 5937
Email Tos present: 5937
Email Froms present: 5937
Email Bodies present: 5937


Excellent, they're al the same length, so now let's create a dict that we can then conver to a Pandas DF

In [8]:
emailsDict = {'ID':email_id, 'To':email_to_list, 'From':email_from_list}

In [9]:
import pandas as pd

In [117]:
email_df = pd.DataFrame.from_dict(emailsDict)

In [11]:
email_df.head()

,From,ID,To
0,rosalee.fleming@enron.com,<18133935.1075840283210.JavaMail.evans@thyme>,sherri.reinartz@enron.com
1,rosalee.fleming@enron.com,<2156358.1075840283423.JavaMail.evans@thyme>,lizard_ar@yahoo.com
2,rosalee.fleming@enron.com,<20840329.1075840285588.JavaMail.evans@thyme>,rob.bradley@enron.com
3,tori.wells@enron.com,<22263156.1075840285610.JavaMail.evans@thyme>,michael@optsevents.com
4,rosalee.fleming@enron.com,<11395510.1075840285634.JavaMail.evans@thyme>,jaime.alatorre@enron.com


Notice that the email in row 19 has multiple addresses on the To line. In these instances, let's create a sub-list there

In [119]:
email_df.loc[19,:]

From                            rosalee.fleming@enron.com
ID          <27268321.1075840286103.JavaMail.evans@thyme>
To      tori.wells@enron.com, vanessa.groscrand@enron....
Name: 19, dtype: object

Also note that some of the records have special characters in there

In [120]:
email_df['To'] = email_df['To'].astype(str) \
    .replace('\n','').replace('\t','').replace(' ','') \
    .str.split(',')

# str.replace('\n','').str.split(',')

In [122]:
email_df.loc[91, ]

From                            rosalee.fleming@enron.com
ID          <23327870.1075840296045.JavaMail.evans@thyme>
To      [herrold@swbell.net,  david@enlineresources.co...
Name: 91, dtype: object

Let's find the emails that are to multiple people

In [69]:
email_df[email_df['To'].str.len() > 1].head()

,From,ID,To
19,rosalee.fleming@enron.com,<27268321.1075840286103.JavaMail.evans@thyme>,"[tori.wells@enron.com, vanessa.groscrand@enro..."
42,rosalee.fleming@enron.com,<2709412.1075840286591.JavaMail.evans@thyme>,"[sherrie@layfam.com, sally.keepers@enron.com]"
91,rosalee.fleming@enron.com,<23327870.1075840296045.JavaMail.evans@thyme>,"[herrold@swbell.net, david@enlineresources.co..."
154,rosalee.fleming@enron.com,<1508214.1075840283772.JavaMail.evans@thyme>,"[joe.hillings@enron.com, cynthia.sandherr@enr..."
189,rosalee.fleming@enron.com,<26171346.1075840284007.JavaMail.evans@thyme>,"[joe.hillings@enron.com, cynthia.sandherr@enr..."


Since there are emails to multiple people we'll need to re-write the function to parse this out.

To whom did Rosalee send the most emails

In [126]:
from_to = []
for index, row in email_df.iterrows():
    for to in row['To']:
        from_to.append([index, row['From'], to \
                        .replace('\n','').replace('\t','').replace(' ','')])

In [129]:
from_rosalee_to = []
from collections import Counter
for item in from_to:
    if (item[1] == 'rosalee.fleming@enron.com'):
        from_rosalee_to.append(item[2])
print(Counter(from_rosalee_to).most_common(10))

[('lizard_ar@yahoo.com', 36), ('jeffrey.garten@yale.edu', 27), ('katherine.brown@enron.com', 26), ('expense.report@enron.com', 25), ('sherri.sera@enron.com', 21), ('sally.keepers@enron.com', 16), ('steven.kean@enron.com', 16), ('joe.hillings@enron.com', 15), ('skimmel@iexalt.net', 14), ('shea_dugger@i2.com', 13)]


# Let's Analyze the Sentiment of Rosalee

In [140]:
baseDir = 'data/enron_emails/maildir/lay-k'

email_id = []
email_to_list = []
email_from_list = []
email_body = []

for directory, subdirectory, filenames in os.walk(baseDir):
    for filename in filenames:
        email_analysis(os.path.join(directory,filename), email_id, email_to_list, email_from_list, email_body)
        
emailsWithBodyDict = {'ID':email_id, 'To':email_to_list, 'From':email_from_list, 'Body':email_body}

In [141]:
emailWithBody_df = pd.DataFrame.from_dict(emailsWithBodyDict)

In [157]:
sentiment_by_employee = []
senders = []
for index, row in emailWithBody_df.iterrows():
    senders.append(row['From'])
    sentiment_by_employee.append(TextBlob(row['Body']).sentiment.polarity)

In [158]:
print(len(sentiment_by_employee))
print(len(senders))

5937
5937


In [160]:
sentDict = {'Sender':senders, 'Sentiment':sentiment_by_employee}
sentiment_df = pd.DataFrame.from_dict(sentDict)

In [166]:
# sentiment_df[sentiment_df['Sender'] == 'rosalee.fleming@enron.com']
sentiment_df[sentiment_df['Sender'] == 'rosalee.fleming@enron.com'].groupby('Sender')['Sentiment'].agg('mean')

Sender
rosalee.fleming@enron.com    0.120996
Name: Sentiment, dtype: float64

# Now let's run it on the full data set

Don't run this on local machine

In [176]:
baseDir = 'data/enron_emails/maildir/lay-k'

email_id = []
email_to_list = []
email_from_list = []
email_body = []

for directory, subdirectory, filenames in os.walk(baseDir):
    for filename in filenames:
        email_analysis(os.path.join(directory,filename), email_id, email_to_list, email_from_list, email_body)
        
emailsWithBodyDict = {'ID':email_id, 'To':email_to_list, 'From':email_from_list, 'Body':email_body}

In [190]:
emailWithBody_df = pd.DataFrame.from_dict(emailsWithBodyDict)
sentiment_by_employee = []
senders = []
for index, row in emailWithBody_df.iterrows():
    parsed_senders = row['From'].replace('<','').replace('>','')
    senders.append(parsed_senders)
    sentiment_by_employee.append(TextBlob(row['Body']).sentiment.polarity)
sentDict = {'Sender':senders, 'Sentiment':sentiment_by_employee}
sentiment_df = pd.DataFrame.from_dict(sentDict)

In [199]:
sentiment_final = sentiment_df.groupby('Sender')['Sentiment'].agg(['mean','max','count'])
sentiment_final[sentiment_final['count'] > 100].sort_values(['mean','count'], ascending=False).head()

,mean,max,count
Sender,,,
rosalee.fleming@enron.com,0.120996,1.0,856


# Cluster Code

In [2]:
emailsRaw = sc.textFile("hdfs://user/systest/datasets/lay-k")


NameError: name 'sc' is not defined